In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from src.config import BATCH_SIZE
import random 
import numpy as np 
import torch 
from src.data_loader import read_data
from src.networks.unimodal_BERT import unimodal_dBERT_Dataset

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)
g = torch.Generator()
g.manual_seed(0)

def get_dataloader(data_type):
    data = read_data(data_type)
    dataset = unimodal_dBERT_Dataset(data)
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE,shuffle=True, worker_init_fn=seed_worker, generator=g)
    return data_loader

In [7]:
train_loader = get_dataloader('train')
val_loader = get_dataloader('val')
test_loader = get_dataloader('test')

100%|██████████| 200/200 [00:00<00:00, 7703.18it/s]


In [8]:
from src.trainer import Trainer
from src.networks.unimodal_BERT import unimodal_dBERT_Model, unimodal_dBERT_Input_transformer
import torch.nn as nn
import torch 

trainer = Trainer(10)

#Set data
trainer.set_data(train_loader, val_loader)

#set model 
model = unimodal_dBERT_Model().cuda()
input_transformer = unimodal_dBERT_Input_transformer()
trainer.set_model(model, input_transformer)

#Backpropagation
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
trainer.set_optimizer(optimizer, loss_fn)
trainer.train()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2/2 [00:00<00:00,  4.58it/s]




For epoch = 0
Training Loss = 0.5012521662495353 | Training Accuracy = 0.05009469696969697
Validation Loss = 0.33975617587566376|Validation Accuracy = 0.015625




100%|██████████| 2/2 [00:00<00:00,  4.59it/s]




For epoch = 1
Training Loss = 0.28957589851184323 | Training Accuracy = 0.09715909090909092
Validation Loss = 0.31735847890377045|Validation Accuracy = 0.03125




100%|██████████| 2/2 [00:00<00:00,  4.57it/s]


...



For epoch = 7
Training Loss = 0.47755799103866925 | Training Accuracy = 0.014393939393939393
Validation Loss = 0.5640276372432709|Validation Accuracy = 0.0




100%|██████████| 2/2 [00:00<00:00,  4.49it/s]




For epoch = 8
Training Loss = 0.40155708518895233 | Training Accuracy = 0.05965909090909091
Validation Loss = 0.4906644970178604|Validation Accuracy = 0.07913306451612903




100%|██████████| 2/2 [00:00<00:00,  4.47it/s]



For epoch = 9
Training Loss = 0.4458889663219452 | Training Accuracy = 0.08579545454545455
Validation Loss = 0.6101725101470947|Validation Accuracy = 0.016129032258064516




In [9]:
trainer.test(test_loader)

100%|██████████| 22/22 [00:04<00:00,  4.45it/s]

Accuracy Score = 0.042212518195050945
Macro F1 score Score = 0.10869658670415396
Micro F1 score Score = 0.37677793904208995
Classification Report


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.33      0.29      0.31        70
           3       0.00      0.00      0.00         0
           4       0.08      1.00      0.15        30
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.41      0.30      0.35        83
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.84      0.13      0.


C:\Users\ADITYA RATHORE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ADITYA RATHORE\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
